# 03 – Local Search Experiments

This notebook explores **local search** methods (hill climbing style)
to improve tree packings for the Santa 2025 – Christmas Tree Packing Challenge.

Starting point:
- A simple **hexagonal lattice baseline** layout for each `n`.

Goals:
- Define a configuration as a list of tree poses `(x, y, angle, polygon)`.
- Implement a hill-climbing local search that:
  * Proposes small random moves and rotations for single trees.
  * Rejects moves that cause overlaps.
  * Accepts moves that reduce the bounding square side.
- Compare **before vs after** side lengths and visualize the layouts.


In [ ]:
import math
from dataclasses import dataclass
from typing import List, Tuple
import random

import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

%matplotlib inline
plt.rcParams['figure.dpi'] = 120
random.seed(1234)
np.random.seed(1234)


In [ ]:
# Tree polygon in local coordinates
# Origin (0, 0) = center of the top of the trunk

TREE_TEMPLATE_VERTS = np.array([
    [0.0, 0.8],          # Tip of the tree
    [0.25 / 2, 0.5],     # Right top tier
    [0.25 / 4, 0.5],
    [0.4 / 2, 0.25],     # Right middle tier
    [0.4 / 4, 0.25],
    [0.7 / 2, 0.0],      # Right bottom tier
    [0.15 / 2, 0.0],     # Right trunk top
    [0.15 / 2, -0.2],    # Right trunk bottom
    [-0.15 / 2, -0.2],   # Left trunk bottom
    [-0.15 / 2, 0.0],    # Left trunk top
    [-0.7 / 2, 0.0],     # Left bottom tier
    [-0.4 / 4, 0.25],    # Left middle tier
    [-0.4 / 2, 0.25],
    [-0.25 / 4, 0.5],    # Left top tier
    [-0.25 / 2, 0.5],
], dtype=float)

TREE_RADIUS = float(np.linalg.norm(TREE_TEMPLATE_VERTS, axis=1).max())
TREE_RADIUS


In [ ]:
def make_tree_polygon(x: float, y: float, angle_deg: float) -> Polygon:
    """Build a tree polygon at (x, y) with rotation angle_deg."""
    theta = math.radians(angle_deg)
    c, s = math.cos(theta), math.sin(theta)
    rot = np.array([[c, -s], [s, c]], dtype=float)
    pts = TREE_TEMPLATE_VERTS @ rot.T
    pts[:, 0] += x
    pts[:, 1] += y
    return Polygon(pts)

@dataclass
class TreePose:
    x: float
    y: float
    angle: float
    poly: Polygon

    @classmethod
    def from_params(cls, x: float, y: float, angle: float) -> "TreePose":
        return cls(x=x, y=y, angle=angle, poly=make_tree_polygon(x, y, angle))

    def update(self) -> None:
        self.poly = make_tree_polygon(self.x, self.y, self.angle)


In [ ]:
def generate_hex_centers(num_points: int, radius: float = TREE_RADIUS) -> List[Tuple[float, float, int, int]]:
    """Generate a hexagonal lattice of centers around the origin.
    Returns (x, y, row, col) sorted by distance to origin.
    """
    dx = 2.0 * radius
    dy = math.sqrt(3.0) * radius
    max_ring = 20
    centers = []
    for row in range(-max_ring, max_ring + 1):
        offset = 0.5 * dx if (row & 1) else 0.0
        for col in range(-max_ring, max_ring + 1):
            x = col * dx + offset
            y = row * dy
            centers.append((x, y, row, col))
    centers.sort(key=lambda c: c[0] * c[0] + c[1] * c[1])
    return centers[:num_points]

HEX_CENTERS = generate_hex_centers(200)
len(HEX_CENTERS), HEX_CENTERS[0]


In [ ]:
def initial_hex_layout_for_n(n: int) -> List[TreePose]:
    """Take the first n hex centers and assign a simple orientation pattern."""
    poses: List[TreePose] = []
    for k in range(n):
        x, y, row, col = HEX_CENTERS[k]
        angle = 0.0 if (row % 2 == 0) else 180.0
        poses.append(TreePose.from_params(x, y, angle))
    return poses

# Quick smoke test
poses_5 = initial_hex_layout_for_n(5)
len(poses_5), poses_5[0]


In [ ]:
def has_any_collision(poses: List[TreePose]) -> bool:
    polys = [p.poly for p in poses]
    index = STRtree(polys)
    for i, poly in enumerate(polys):
        candidates = index.query(poly)
        for j in candidates:
            if i == j:
                continue
            if poly.intersects(polys[j]) and not poly.touches(polys[j]):
                return True
    return False

def bounding_square_side(poses: List[TreePose]) -> float:
    union = unary_union([p.poly for p in poses])
    minx, miny, maxx, maxy = union.bounds
    width, height = maxx - minx, maxy - miny
    return max(width, height)

def plot_layout(poses: List[TreePose], title: str = "", figsize=(6, 6)):
    union = unary_union([p.poly for p in poses])
    minx, miny, maxx, maxy = union.bounds
    side = max(maxx - minx, maxy - miny)
    fig, ax = plt.subplots(figsize=figsize)
    for p in poses:
        xs, ys = p.poly.exterior.xy
        ax.fill(xs, ys, alpha=0.4)
        ax.plot(xs, ys, linewidth=0.8)
    ax.add_patch(
        plt.Rectangle((minx, miny), side, side,
                      fill=False, edgecolor='red', linestyle='--', linewidth=2)
    )
    pad = 0.5
    ax.set_xlim(minx - pad, minx + side + pad)
    ax.set_ylim(miny - pad, miny + side + pad)
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')
    ax.set_title(f"{title} (side≈{side:.4f})")
    plt.show()

print('n=5 collision?', has_any_collision(poses_5))
plot_layout(poses_5, title='Initial hex layout, n=5')


In [ ]:
def refine_layout(
    poses: List[TreePose],
    iters: int = 2000,
    move_scale: float = 0.02,
    angle_scale: float = 2.0,
) -> float:
    """Hill-climbing local search.

    - Randomly picks a tree each iteration.
    - Proposes a small (dx, dy) and d_angle.
    - Rejects any move that causes overlaps.
    - Accepts moves that decrease the bounding square side.
    Returns the final best side length.
    """
    best_side = bounding_square_side(poses)
    for step in range(iters):
        idx = random.randrange(len(poses))
        tree = poses[idx]

        old_x, old_y, old_angle = tree.x, tree.y, tree.angle
        old_poly = tree.poly

        dx = move_scale * random.gauss(0.0, 1.0)
        dy = move_scale * random.gauss(0.0, 1.0)
        d_angle = angle_scale * random.gauss(0.0, 1.0)

        tree.x += dx
        tree.y += dy
        tree.angle += d_angle
        tree.update()

        # Collision check only for this tree vs others
        others = [p.poly for i, p in enumerate(poses) if i != idx]
        index = STRtree(others)
        cand = index.query(tree.poly)
        collision = any(
            tree.poly.intersects(others[j]) and not tree.poly.touches(others[j])
            for j in cand
        )

        if collision:
            tree.x, tree.y, tree.angle = old_x, old_y, old_angle
            tree.poly = old_poly
            continue

        new_side = bounding_square_side(poses)
        if new_side <= best_side:
            best_side = new_side
        else:
            tree.x, tree.y, tree.angle = old_x, old_y, old_angle
            tree.poly = old_poly

    return best_side


In [ ]:
n = 50
poses_base = initial_hex_layout_for_n(n)
print('Baseline collision?', has_any_collision(poses_base))
side_base = bounding_square_side(poses_base)
print(f'Baseline side (n={n}): {side_base:.6f}')
plot_layout(poses_base, title=f'Baseline hex layout, n={n}')

# Copy poses for refinement
import copy
poses_refined = copy.deepcopy(poses_base)
side_final = refine_layout(poses_refined, iters=4000, move_scale=0.02, angle_scale=1.5)
print(f'Refined side (n={n}): {side_final:.6f}')
improvement = side_base - side_final
print(f'Improvement: {improvement:.6f}')
plot_layout(poses_refined, title=f'Refined layout, n={n}')


In [ ]:
def run_summary(ns, iters_per_n=3000):
    rows = []
    for n in ns:
        base = initial_hex_layout_for_n(n)
        side_base = bounding_square_side(base)
        poses = [TreePose.from_params(p.x, p.y, p.angle) for p in base]
        side_ref = refine_layout(poses, iters=iters_per_n)
        rows.append((n, side_base, side_ref, side_base - side_ref))
    return rows

ns_list = [5, 10, 20, 50, 100, 200]
rows = run_summary(ns_list, iters_per_n=2000)
print(' n   side_base   side_ref    improvement')
for n, sb, sr, imp in rows:
    print(f"{n:3d}  {sb:9.4f}  {sr:9.4f}   {imp:9.4f}")
